<a href="https://colab.research.google.com/github/manuelarguelles/Speech-to-Text-Transcription-Tool-v2/blob/main/20240625_Whisper_transcribir_traducir_2_interlocutores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transcripción y traducción WHISPER V 0.1

1. Este cuaderno permite hacer una transcripción o una traducción de un
audio o video en la mayoria de formatos populares en cualquier idioma o traducirlo al inglés (o ambas tareas). El resultado son archivos de texto (txt, srt y vtt) planos y con los tiempos para usar como subtitulos. Los formatos soportados son los siguientes https://en.wikipedia.org/wiki/FFmpeg#Supported_codecs_and_formats.
1. Los archivos de audio y los archivos de texto generados quedarán almacenados en tu google drive en una carpeta llamada whisper, que se borrará y recreará con cada nueva ejecución.
1. Este cuaderno NO codifica video ni audio, solo genera los archivos de texto preparados para susbtítulos.

# Instrucciones de uso:

1. El entorno de ejecución deberá tener aceleración por hardware de GPU. (La capa gratuita T4 sería suficiente)
1. Se deben ejecutar las tres tareas secuencialmente.
1. Una vez ejecutadas las tres tareas, en consecuentes ejecuciones solo debemos ejecutar las tareas 2 y 3.
1. Con la primera tarea instalamos Whisper y las librerías necesarias, además de dar permiso a nuestro Drive para poder guardar los archivos.
1. Con la tarea 2 seleccionamos el archivo de audio a utilizar y los parámetros a utilizar en la tarea (ver los parámetros de la tarea 2).
1. Con la tarea 3 realizamos la transcripción/traducción y copia de archivos a su destino definitivo en nuestro Google Drive.

# Audio Mp3 Demo:

Para este demo se utilizó el audio .mp3 de la entrevista a Julian Assange:
https://www.youtube.com/watch?v=HNOnvp5t7Do

Para descargar vídeos de youtube a mp3:
https://y2meta.mobi/download/

Este audio en .mp3 se encuentra en esta ruta onedrive:
https://1drv.ms/u/s!AhhQzA6574kSkuQW89G_oJnD9ecv5w?e=9Fqci8

---




In [1]:
#@title 1) Instalamos las dependencias y librerías y montamos el Google Drive.

# Install Whisper
!pip install git+https://github.com/openai/whisper.git > /dev/null 2>&1
!pip install jiwer whisper > /dev/null 2>&1

# Mount Google Drive.
g_drive = '/content/My_Gdrive'
from google.colab import drive
drive.mount( g_drive )

Drive already mounted at /content/My_Gdrive; to attempt to forcibly remount, call drive.mount("/content/My_Gdrive", force_remount=True).


### 1. En primer lugar deberemos seleccionar los tres siguientes parámetros (o dejar las opciones por defecto).

**task:** L tarea que va a realizarse, traducir, transcribir o ambas.

**model:** Modelo de whisper usado para hacer la tarea, tiny y small son los menos complejos y más rápidos, large es el más complejo y lento, con el que se lograra mejores resultados en general.

**langauge:**  Idioma del audio a transcribir/traducir, que puede ser Español (*Spanish*), Inglés (*Inglés*) o Autodetectar (*Autodetect*).

### 2. A continuación debermos ejecutar la tarea, lo que nos permitirá elegir el archivo de audio para transcribir/convertir en el siguinete paso.

In [2]:
import os
from google.colab import files
import shutil

# Montar Google Drive
g_drive = '/content/My_Gdrive'
from google.colab import drive
drive.mount(g_drive)

# Definir nombres de carpetas
folder_name = g_drive + '/MyDrive/whisper'
folder_name_en = g_drive + '/MyDrive/whisper/en'

# Si la carpeta existe, la eliminamos
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)

# Crear carpetas
os.mkdir(folder_name)
os.mkdir(folder_name_en)

# Parámetros configurables
task = "Transcripcion"  #@param ["Transcription and Translation", "Translation", "Transcripcion"]
model = "large"  #@param ["large", "medium", "tiny", "tiny.en", "base.en", "base", "small.en", "small", "medium.en", "large-v1", "large-v2"]
language = "English"  #@param ["Autodetect", "Spanish", "English"]
num_interlocutors = 2  #@param [1, 2]  # Número de interlocutores

if language == 'Autodetect':
    language_param = ''
else:
    language_param = '--language ' + language

# Subir archivos
uploaded_files = files.upload()
for file in uploaded_files.keys():
    # Normalizar nombre de archivo
    file2 = file.replace(' ', '').replace(')', '').replace('(', '')
    destination = os.path.join(folder_name, file2)
    print(f'Moviendo {file} a {file2}')
    shutil.copy(file, destination)
    extension = os.path.splitext(file2)[1]
    file_name = os.path.splitext(file2)[0]
    print(f'{file_name}{extension}')

    # Convertir a MP3 si no lo es
    if extension != '.mp3':
        !ffmpeg -i {folder_name}/{file_name}{extension} {folder_name}/{file_name}.mp3 > /dev/null 2>&1
        destination_new = folder_name + '/' + file_name + '.mp3'
        !rm {destination} > /dev/null 2>&1
    else:
        destination_new = destination

# Proceso de transcripción y traducción según el número de interlocutores
if num_interlocutors == 1:
    # Código para un interlocutor
    print("Procesando con un interlocutor...")
    !whisper {destination_new} --model {model} {language_param} --output_dir {folder_name_en} > /dev/null 2>&1
elif num_interlocutors == 2:
    # Código para dos interlocutores
    print("Procesando con dos interlocutores...")
    !whisper {destination_new} --model {model} {language_param} --output_dir {folder_name_en} --dual_channel > /dev/null 2>&1


Drive already mounted at /content/My_Gdrive; to attempt to forcibly remount, call drive.mount("/content/My_Gdrive", force_remount=True).


Saving Julian Assange_ Why the world needs WikiLeaks (320).mp3 to Julian Assange_ Why the world needs WikiLeaks (320).mp3
Moviendo Julian Assange_ Why the world needs WikiLeaks (320).mp3 a JulianAssange_WhytheworldneedsWikiLeaks320.mp3
JulianAssange_WhytheworldneedsWikiLeaks320.mp3
Procesando con un interlocutor...


### 3. Ejecutamos la tarea encargada de realizar la transcripción/traducción.

In [3]:
#@title 3) Ejecutar Whisper con los parámetros seleccionados

def cl_transcript():
  print( 'Se va a realizar una transcripción', "\n" )
  print( '-----------------------------------------', "\n")

  # Execute whisper command.
  !whisper {destination_new} --task transcribe {language_param} --model {model}

  # Move generated files to GDrive folder.
  !mv /content/*.srt {folder_name}
  !mv /content/*.txt {folder_name}
  !mv /content/*.vtt {folder_name}

  print( "\n\n" )
  print( '-----------------------------------------', "\n")
  print( 'los archivos de la transcripción fueron copiados en ',  folder_name )
  print( "\n\n" )

def cl_translate():
  print( 'Se va a realizar una traducción', "\n" )
  print( '-----------------------------------------', "\n")

  # Execute whisper command.
  !whisper {destination_new} --task translate {language_param} --model {model}

  # Move generated files to GDrive folder and subfolder for English.
  !mv /content/*.srt {folder_name_en}
  !mv /content/*.txt {folder_name_en}
  !mv /content/*.vtt {folder_name_en}

  # Rename files to en-*.*
  !mv {folder_name_en}/{file_name}.srt {folder_name_en}/en-{file_name}.srt
  !mv {folder_name_en}/{file_name}.txt {folder_name_en}/en-{file_name}.txt
  !mv {folder_name_en}/{file_name}.vtt {folder_name_en}/en-{file_name}.vtt

  print( "\n\n" )
  print( '-----------------------------------------', "\n")
  print( 'los archivos de la traducción fueron copiados en ',  folder_name_en )
  print( "\n\n" )

if task == "Transcripcion":
  cl_transcript()
elif task == "Translation":
  cl_translate()
else:
  cl_transcript()
  cl_translate()


Se va a realizar una transcripción 

----------------------------------------- 

[00:00.000 --> 00:15.960]  Julián, bienvenido.
[00:16.600 --> 00:26.040]  Se ha reportado que Wikileaks, tu bebé, en los últimos años ha lanzado más documentos clasificados que el resto de los medios mundiales.
[00:26.600 --> 00:28.120]  ¿Puede ser verdad?
[00:28.120 --> 00:31.520]  Sí, ¿puede ser verdad? Es una preocupación, ¿no es?
[00:31.760 --> 00:37.800]  El resto de los medios mundiales están haciendo un trabajo tan malo que un pequeño grupo de activistas
[00:37.800 --> 00:42.920]  es capaz de lanzar más de ese tipo de información que el resto de la prensa mundial combinada.
[00:43.240 --> 00:50.700]  ¿Cómo funciona? ¿Cómo lanzan los documentos y cómo aseguran su privacidad?
[00:51.640 --> 00:55.440]  Estos son, por lo que podemos ver, los clásicos explosores.
[00:55.440 --> 00:58.480]  Y tenemos un montón de maneras para que nos den información.
[00:59.480 --> 01:02.620]  Así que usamos esta encript